# Importera rensade RBX från dgn till gdb
Gör på detta sätt även 2022

In [15]:
# Copy RBX from dgn to gdb feature class, one fc for each LGXXX_Y
##############PROBLEM 2/12: THE POINTS IMPORTED 
import arcpy, os, glob, SVK
from pathlib import Path

LG_line_dict = {6:[1,2], 9:[1]} #

prj_dir = r"Z:\Systemutveckling\Data\Indata\Exempeldata_2021"
dgn_dir = os.path.join(prj_dir, "DGN")
gdb = os.path.join(prj_dir, "SVK_2021_GIS", "SVK_2021_GIS.gdb")

output_fd = os.path.join(gdb, "clean_RBX_points")


def sweref99TM():
    wkt = 'PROJCS["SWEREF99_TM",GEOGCS["GCS_SWEREF99",DATUM["D_SWEREF99",SPHEROID["GRS_1980",6378137,298.257222101]],PRIMEM["Greenwich",0],UNIT["Degree",0.017453292519943295]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",15],PARAMETER["scale_factor",0.9996],PARAMETER["false_easting",500000],PARAMETER["false_northing",0],UNIT["Meter",1]]'
    spatialref = arcpy.SpatialReference()
    spatialref.loadFromString(wkt)
    return(spatialref)

sr = sweref99TM()


for LG_nr in LG_line_dict:
    os.chdir(dgn_dir)
    LG = f'LG{str(LG_nr).zfill(3)}'
    print(LG)
    
    for line in LG_line_dict[LG_nr]:
        dgn = os.path.join(dgn_dir, LG+"_"+str(line)+".dgn") #
        print(dgn)
        
        output_fc = os.path.join(output_fd, LG+"_"+str(line)+"_RBX_clean") #var punkterna ska exporteras till
        
        # Create a value table that will hold the input feature classes for Merge
        vTab = arcpy.ValueTable()
        print("vTab created")
        layername_RBX = dgn + '_Layer_LVL20' #RBX ligger i lager nr 20, typ lagernamn i AC.
        # Behöver det vara unikt för varje del-ledning (line)? Fixa en try/except eller liknande som rensar 
        # bort detta lager vid krasch. Om det inte tas bort kommer det att ligga kvar och hindra cellen från att köra nästa gång
        arcpy.MakeFeatureLayer_management(dgn + '/Point', layername_RBX, "\"Level\" = 20")
        print("MakeFeatureLayer complete")
        vTab.addRow(layername_RBX)
        print("Row added to vTab")
          
    # Merge the CAD features into one feature class
    arcpy.env.workspace = output_fd
    arcpy.Merge_management(vTab, output_fc)
    print("vTab merged to output_fc")
    
    arcpy.Delete_management(layername_RBX)
    print("Layer deleted")

    arcpy.env.workspace = dgn_dir
    

LG006
Z:\Systemutveckling\Data\Indata\Exempeldata_2021\DGN\LG006_1.dgn
vTab created
MakeFeatureLayer complete
Row added to vTab
Z:\Systemutveckling\Data\Indata\Exempeldata_2021\DGN\LG006_2.dgn
vTab created
MakeFeatureLayer complete
Row added to vTab
vTab merged to output_fc
Layer deleted
LG009
Z:\Systemutveckling\Data\Indata\Exempeldata_2021\DGN\LG009_1.dgn
vTab created
MakeFeatureLayer complete
Row added to vTab
vTab merged to output_fc
Layer deleted


In [14]:
arcpy.Delete_management("Z:\Systemutveckling\Data\Indata\Exempeldata_2021\DGN\LG006_1.dgn_Layer_LVL20")
arcpy.Delete_management("Z:\Systemutveckling\Data\Indata\Exempeldata_2021\DGN\LG006_2.dgn_Layer_LVL20")
arcpy.Delete_management("Z:\Systemutveckling\Data\Indata\Exempeldata_2021\DGN\LG009_1.dgn_Layer_LVL20")

<Result 'true'>

# Near 3D - avstånd till fas för RBX

In [16]:
# This is where actual computations begin, previous steps are for arranging data
# This version of the script reads RBX points from a 3D point feature class in an ESRI file geodatabase (gdb)
# We can assume that future scripts will read RBX points from txt files: one file per TerraScan block,
# with X, Y, Z, dZ for each point.

import arcpy, os, glob, SVK
arcpy.CheckOutExtension('3D')
from pathlib import Path
import pandas as pd

LGs_info_path = r"Z:\Systemutveckling\Data\Indata\Exempeldata_2021\ledningar_2021.txt"
#gdb = r"Z:\26664\6605189\000\Arbetsmapp_analys\SVK_2021\SVK_2021_GIS\SVK_2021_GIS.gdb"
gdb = r"Z:\Systemutveckling\Data\Indata\Exempeldata_2021\SVK_2021_GIS\SVK_2021_GIS.gdb"
fd_RBX = os.path.join(gdb, 'clean_RBX_points')
fd_wires = os.path.join(gdb, 'Wires')


def dist2wire(fd_RBX, fc_name_RBX, fc_wires, radius):
    # Compute distance from RBX point to wire
    fc_RBX = os.path.join(fd_RBX, fc_name_RBX)
    arcpy.Near3D_3d(in_features=fc_RBX, 
                    near_features=fc_wires, 
                    search_radius=radius, 
                    location="NO_LOCATION", 
                    angle="NO_ANGLE", 
                    delta="NO_DELTA")
    print('done with Near3D '+fc_name_RBX)
    arcpy.DeleteField_management(in_table=fc_RBX, drop_field="NEAR_FID")
    arcpy.DeleteField_management(in_table=fc_RBX, drop_field="NEAR_DIST") # Vi ska redovisa horisontellt avstånd 2022 - behåll detta fält eller beräknas det i RBX_polygons.py?
    arcpy.management.AlterField(in_table=fc_RBX, field="NEAR_DIST3", new_field_name="AVST_FAS", new_field_alias="AVST_FAS")
    print('done with delete and alter field '+fc_name_RBX)
    # Diverse ändringar i DeleteField och AlterField som Karin ska skicka upp på GitHub
    
    
def RBX_3D_distance(powerline):
    LG = powerline[0]
    line = powerline[1]
    littera = powerline[4]
    
    print(f"LG är {LG}, linje är {line}, littera är {littera}")
    
    fc_name_RBX = f"{LG}_{line}_RBX_clean"
    fc_name_wires = f"{LG}_{line}_fas"
    fc_wires = os.path.join(fd_wires, fc_name_wires)
    
    if arcpy.Exists(os.path.join(fd_RBX, fc_name_RBX)):
        dist2wire(fd_RBX, fc_name_RBX, fc_wires, 6)
    else:
        print(f"{os.path.join(fd_RBX, fc_name_RBX)} does not exist")

# The file pointed at by LGs_info_path contains only the powerlines that should be
# processed. So this file has to be updated each time we want to process a new set of
# powerlines. This is unpractical - consider a user prompt where the user can select
# which powerlines to process

LGs_info = pd.read_csv(LGs_info_path, sep="\t", header=0)
LGs_info.apply(RBX_3D_distance, axis=1)


LG är LG006, linje är 1, littera är UL29
Z:\Systemutveckling\Data\Indata\Exempeldata_2021\SVK_2021_GIS\SVK_2021_GIS.gdb\clean_RBX_points\LG006_1_RBX_clean does not exist
LG är LG006, linje är 2, littera är UL6 S2
done with Near3D LG006_2_RBX_clean
done with delete and alter field LG006_2_RBX_clean
LG är LG009, linje är 1, littera är UL1 S1-3
done with Near3D LG009_1_RBX_clean
done with delete and alter field LG009_1_RBX_clean


0    None
1    None
2    None
dtype: object

# Kör RBX_polygon.py innan nästa cell körs
### RBX_polygon.py gör polygoner av RBX-punkterna och delar in dem i "farlighetsklass", sätter minsta avstånd till fas för varje polygon osv

# Put RBX shape polygons in gdb

In [3]:
import arcpy, os, glob, SVK
from pathlib import Path
import pandas as pd

LGs_info_file = "ledningar_2021_RBX_shp2gdb.txt"
prj_dir = r"Z:\26664\6605189\000\Arbetsmapp_analys\SVK_2021"
lev_dir = r"Z:\26664\6605189\000\Projekt\Utleveranser\Skannat_2021"
RBX_shp_dir = os.path.join(prj_dir,"RBX_polygons")
RBX_fd = os.path.join(lev_dir,"Leverans_2022-02-11","Leverans_2022-02-11.gdb","RBX_polygoner")

LGs_info_path = os.path.join(prj_dir, LGs_info_file)

def shp2gdb(powerline):
    LG = powerline[0]
    line = powerline[1]
    
    RBX_shp_path = os.path.join(RBX_shp_dir,f"{LG}_{line}_RBX.shp")
    RBX_fc = f"RBX_{LG}_{line}"
    
    if os.path.exists(RBX_shp_path):
        print(f"Converting {LG}_{line}")
        arcpy.conversion.FeatureClassToFeatureClass(RBX_shp_path, RBX_fd, RBX_fc)
    else:
        print(f"No RBX for {LG}_{line}")

LGs_info = pd.read_csv(LGs_info_path, sep="\t", header=0)
LGs_info.apply(shp2gdb, axis=1)

Converting LG661_1
Converting LG661_2


0    None
1    None
dtype: object

# Merge RBX polygons

In [4]:
# FÖRST: Lägg RBX-polygoner i datummärkt leverans-gdb

LGs_info_file = "ledningar_2021_sample.txt"

prj_dir = r"Z:\26664\6605189\000\Arbetsmapp_analys\SVK_2021"
RBX_fd = os.path.join(prj_dir,"SVK_2021_GIS","SVK_2021_GIS.gdb","RBX_polygons")

leverans_gdb = r"Z:\26664\6605189\000\Projekt\Utleveranser\Skannat_2021\Leverans_2022-01-24\Leverans_2022-01-24.gdb"
leverans_fd = os.path.join(leverans_gdb, "RBX_polygons")

LGs_info_path = os.path.join(prj_dir, LGs_info_file)

def copyRBX(powerline):
    LG = powerline[0]
    line = powerline[1]
    
    RBX_src = os.path.join(RBX_fd, f"RBX_{LG}_{line}")
    RBX_dst = os.path.join(leverans_fd, f"RBX_{LG}_{line}")
    
    print(f"Copying {LG}_{line}")
    arcpy.management.CopyFeatures(RBX_src, RBX_dst)
    print("Done")

LGs_info = pd.read_csv(LGs_info_path, sep="\t", header=0)
LGs_info.apply(copyRBX, axis=1)

Copying LG006_2
Done
Copying LG006_4
Done
Copying LG006_6
Done
Copying LG011_1
Done
Copying LG014_3
Done
Copying LG027_1
Done
Copying LG037_5
Done
Copying LG042_1
Done
Copying LG043_4
Done
Copying LG044_2
Done
Copying LG044_3
Done
Copying LG044_5
Done
Copying LG060_2
Done
Copying LG061_1
Done
Copying LG068_2
Done
Copying LG068_4
Done
Copying LG068_6
Done
Copying LG068_8
Done
Copying LG082_2
Done
Copying LG082_4
Done
Copying LG082_5
Done
Copying LG083_1
Done
Copying LG093_1
Done
Copying LG115_1
Done
Copying LG115_2
Done
Copying LG122_2
Done
Copying LG122_3
Done


0     None
1     None
2     None
3     None
4     None
5     None
6     None
7     None
8     None
9     None
10    None
11    None
12    None
13    None
14    None
15    None
16    None
17    None
18    None
19    None
20    None
21    None
22    None
23    None
24    None
25    None
26    None
dtype: object

In [2]:
gdb = r"Z:\26664\6605189\000\Projekt\Utleveranser\Skannat_2021\Leverans_2022-01-24\Leverans_2022-01-24.gdb"

outFcName = "RBX24jan"

inDB = gdb + "\\" + "RBX_polygons"
outDB = gdb

arcpy.env.workspace= inDB

polyList = list()

for featClass in arcpy.ListFeatureClasses():
#for featClass in fcNameList:
    print(featClass)
    desc = arcpy.Describe(inDB + "\\" + featClass)
    if desc.shapeType == "Polygon":
        polyList.append(inDB + "\\" + featClass)


In [13]:
print(outDB + "\\" + outFcName)

Z:\26664\6605189\000\Projekt\Utleveranser\Skannat_2021\Leverans_2021-01-20\Leverans_2022-01-20.gdb\RBX20jan


In [14]:
if len(polyList) > 0:
    arcpy.Merge_management(polyList, outDB + "\\" + outFcName)

In [21]:
# Copy RBX from dgn to gdb feature class, one fc for each LGXXX_Y
##############PROBLEM 2/12: THE POINTS IMPORTED 
import arcpy, os, glob, SVK
from pathlib import Path

LG_line_dict = {28:[5]}

prj_dir = r"C:\Users\SEKNNO\OneDrive - Sweco AB\SVK_2022\Testleverans_LG028"
dgn_dir = r"C:\Users\SEKNNO\OneDrive - Sweco AB\SVK_2022\Testleverans_LG028\LG028"
gdb = r"C:\Users\SEKNNO\OneDrive - Sweco AB\SVK_2022\SVK_2022_GIS\SVK_2022_GIS.gdb"

output_fd = os.path.join(gdb, "RBX_test_deleteme")
output_fc = "DELETEME_RBX_LG028_5_clean"

def sweref99TM():
    wkt = 'PROJCS["SWEREF99_TM",GEOGCS["GCS_SWEREF99",DATUM["D_SWEREF99",SPHEROID["GRS_1980",6378137,298.257222101]],PRIMEM["Greenwich",0],UNIT["Degree",0.017453292519943295]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",15],PARAMETER["scale_factor",0.9996],PARAMETER["false_easting",500000],PARAMETER["false_northing",0],UNIT["Meter",1]]'
    spatialref = arcpy.SpatialReference()
    spatialref.loadFromString(wkt)
    return(spatialref)

sr = sweref99TM()

# Create a value table that will hold the input feature classes for Merge
layer_RBX = "Layer_RBX"
arcpy.Delete_management(layer_RBX)

vTab_RBX = arcpy.ValueTable()
print("vTab created")

arcpy.MakeFeatureLayer_management(dgn + '/Point', layer_RBX, "\"Level\" = 5")
print("MakeFeatureLayer complete")
vTab_RBX.addRow(layer_RBX)
print("Row added to vTab_RBX")

# Merge the CAD features into one feature class
arcpy.env.workspace = output_fd
arcpy.Merge_management(vTab_RBX, output_fc)
print("vTab merged to output_fc")

arcpy.Delete_management(layer_RBX)
print("Layer deleted")
arcpy.env.workspace = dgn_dir

vTab created
MakeFeatureLayer complete
Row added to vTab_RBX
vTab merged to output_fc
Layer deleted


In [16]:
vTab.rowCount

0